# HW1 SOLUTION : [PRAVALIKA_AKKURI | 999903872]

### (80%) Perform basic data engineering on the light pollution dataset.


For our data engineering we want to broadly
complete the following:

1. combine all the GaN files into a single file and thus a
   single Pandas DataFrame,
2. convert specific data fields to numeric fields using
   Pandas,
3. combine date and time fields for easier time series evaluation,
3. split data out of specific fields and into new simpler
   fields,
4. use Pandas to understand the underlying distrubution
   of certain data.




**&#167; Task:**  **Use the file of URLs from the first assignment to load the data from 2014-2024 and store it all into a single file.**


In [13]:
import pandas as pd
import glob

# Find all GaN CSVs in your folder (adjust path if needed)
files = glob.glob("sample_data/GaN*.csv")

frames = [pd.read_csv(f) for f in files]
df = pd.concat(frames, ignore_index=True)

# Remove duplicates + reset index (required by assignment)
df = df.drop_duplicates().reset_index(drop=True)

# Save combined file
df.to_csv("2014_to_2024_gan_data.csv", index=False)
print("Combined shape:", df.shape)


Combined shape: (14373, 17)


In [14]:
def assign_location(d):
    """
    Reduces a string to one of four categories, depending on whether "urban", "suburban", "rural" or "remote" is in the string.

    Args:
        d (str): A string or sentence containing one or more of the target words.

    Returns:
        str: the string target or None if not foun
    """
    import re

    d = str(d).lower()
    re_urban = r"\w*(?<![Ss]ub)urban"

    if re.match(re_urban, d):
        return "urban"
    elif "suburban" in d:
        return "suburban"
    elif "rural" in d:
        return "rural"
    elif "remote" in d:
        return "remote"

In [15]:
# CODE SOLUTION HERE
def assign_location(d):
    """
    Map a free-text location comment to one of:
    'urban', 'suburban', 'rural', 'remote', or 'unknown'.
    """
    if pd.isna(d):
        return "unknown"

    s = str(d).lower()

    if "suburban" in s:
        return "suburban"
    elif "urban" in s:
        return "urban"
    elif "rural" in s:
        return "rural"
    elif "remote" in s:
        return "remote"
    else:
        return "unknown"


**&#167; Task:**  **Convert categorical fields to numeric ones.**

We will convert these to numeric values using the following guidance:

* "clear" &#8594; 0
* "1/4 of sky" &#8594; 0.25
* "1/2 of sky" &#8594; 0.50
* "over 1/2 of sky" &#8594; 0.75

In [16]:
# CODE SOLUTION HERE
map_cloud = {
    "clear": 0,
    "1/4 of sky": 0.25,
    "1/2 of sky": 0.50,
    "over 1/2 of sky": 0.75
}

df["CloudCoverPct"] = df["CloudCover"].map(map_cloud)


**&#167; Task:**  **Convert `Constellation` to numeric binary values (binarize the column).**


In [17]:
# CODE SOLUTION HERE
df = pd.get_dummies(df, columns=["Constellation"], prefix="const")


**&#167; Task:**  **Create 5 binary columns `loc_urban`, `loc_suburban`, `loc_rural`, `loc_remote` and `loc_unknown`.**

In [18]:
# CODE SOLUTION HERE
# Apply mapping
df["loc"] = df["LocationComment"].apply(assign_location).fillna("unknown")

# Create binary columns
df = pd.get_dummies(df, columns=["loc"], prefix="", prefix_sep="")

print(df[["urban","suburban","rural","remote","unknown"]].head())


   urban  suburban  rural  remote  unknown
0   True     False  False   False    False
1   True     False  False   False    False
2   True     False  False   False    False
3  False      True  False   False    False
4  False     False  False   False     True


**&#167; Task:**  **Merge `LocalDate` and `LocalTime` and convert to a `datetime` object.**


In [19]:
# CODE SOLUTION HERE
df["LocalDateTime"] = pd.to_datetime(
    df["LocalDate"].astype(str) + " " + df["LocalTime"].astype(str),
    errors="coerce"
)


**&#167; Task:**  **Convert missing `SQMReading` values to `-1`.**

In [20]:
# CODE SOLUTION HERE
df["SQMReading"] = pd.to_numeric(df["SQMReading"], errors="coerce").fillna(-1)


**&#167; Task:**  **Store a new dataset file with a reduced set columns which include our numeric ones.**

In [21]:
# CODE SOLUTION HERE
cols = [
    "Latitude","Longitude","Elevation(m)","LocalDateTime","LimitingMag","SQMReading","CloudCoverPct",
    "const_Bootes","const_Canis Major","const_Crux","const_Cygnus","const_Gemini","const_Grus",
    "const_Hercules","const_Leo","const_None","const_Orion","const_Pegasus","const_Perseus",
    "const_Sagittarius","const_Scorpius","const_Taurus",
    "remote","rural","suburban","unknown","urban"
]

for c in cols:
    if c not in df.columns: df[c] = 0

df[cols].to_csv("2014_to_2024_gan_data_working.csv", index=False)
print("Wrote working file with shape:", df[cols].shape)


Wrote working file with shape: (14373, 27)


BONUS Task: Step (normalize column names)

In [22]:
# ===== BONUS: Normalize column names =====
df_final = df.copy()

# Make names lowercase, replace spaces with "_", remove brackets
df_final.columns = (
    df_final.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("(", "", regex=False)
    .str.replace(")", "", regex=False)
)

# Save the final dataset
df_final.to_csv("2014_to_2024_gan_data_final.csv", index=False)
print("Wrote final file with shape:", df_final.shape)

df_final.head()


Wrote final file with shape: (14373, 38)


,id,obstype,latitude,longitude,elevationm,localdate,localtime,utdate,uttime,limitingmag,...,const_sagittarius,const_scorpius,remote,rural,suburban,unknown,urban,localdatetime,const_none,const_taurus
0,335973,GAN,45.494345,15.533148,110.85,2024-07-27,22:33,2024-07-27,20:33,2,...,False,False,False,False,False,False,True,2024-07-27 22:33:00,0,0
1,335972,GAN,45.499866,15.525713,111.14,2024-07-27,22:25,2024-07-27,20:25,2,...,False,False,False,False,False,False,True,2024-07-27 22:25:00,0,0
2,335971,GAN,45.486892,15.538268,113.34,2024-07-27,22:24,2024-07-27,20:24,2,...,False,False,False,False,False,False,True,2024-07-27 22:24:00,0,0
3,335970,GAN,45.513186,15.521186,114.90,2024-07-27,22:20,2024-07-27,20:20,3,...,False,False,False,False,True,False,False,2024-07-27 22:20:00,0,0
4,335939,GAN,45.495691,15.467396,153.39,2024-07-27,22:31,2024-07-27,20:31,6,...,False,False,False,False,False,True,False,2024-07-27 22:31:00,0,0
